# Installating Julia/IJulia 

### 1 - Downloading and Installing the right Julia binary in the right place

In [ ]:
import os
import sys
import io
import re

In [ ]:
import urllib.request as request  # Python 3

In [ ]:
# get latest stable release info, download link and hashes
g = request.urlopen("https://julialang.org/downloads/")
s = g.read().decode()
g.close;

r = r'<a href=".current_stable_release">([^<]+)</a></h2> ' + \
    r'<p>Checksums for this release are available in both <a href="([^"]*)">MD5</a> and <a href="([^"]*)">SHA256</a> formats.</p>' + \
    r'[^W]*Windows <a href="/downloads/platform/.windows">.help.</a> <td colspan=3 > <a href="[^"]*">64-bit .installer.</a>, <a href="([^"]*)">64-bit .portable.</a>' + \
    r' <td colspan=3 > <a href="[^"]*">32-bit .installer.</a>, <a href="([^"]*)">32-bit .portable.</a>'

release_str, md5link, sha256link, ziplink64bit, ziplink32bit  = re.findall(r,s)[0]
julia_version=re.findall(r"v([^\s]+)",release_str)[0]
print(release_str)
print(julia_version)
print(ziplink64bit)
print(ziplink32bit)
print(md5link)
print(sha256link)

overwrite links, since v1.5.3 installation does not work properly due to
https://github.com/JuliaLang/julia/issues/38411

In [ ]:
if julia_version=='1.5.3':
    julia_version='1.6.0-rc1'
    ziplink64bit='https://julialang-s3.julialang.org/bin/winnt/x64/1.6/julia-1.6.0-rc1-win64.zip'
    md5link='https://julialang-s3.julialang.org/bin/checksums/julia-1.6.0-rc1.md5'
    sha256link='https://julialang-s3.julialang.org/bin/checksums/julia-1.6.0-rc1.sha256'
    print(julia_version)

In [ ]:
# download checksums
g = request.urlopen(md5link)
md5hashes = g.read().decode()
g.close;

g = request.urlopen(sha256link)
sha256hashes = g.read().decode()
g.close;

In [ ]:
# downloading julia (may take 1 minute or 2)

if 'amd64' in sys.version.lower():
    julia_zip=ziplink64bit.split("/")[-1]
    julia_url=ziplink64bit
else:
    julia_zip=ziplink32bit.split("/")[-1]
    julia_url=ziplink32bit
    
hashes=(re.findall(r"([0-9a-f]{32})\s"+julia_zip, md5hashes)[0] , re.findall(r"([0-9a-f]{64})\s+"+julia_zip, sha256hashes)[0])
    
julia_zip_fullpath = os.path.join(os.environ["WINPYDIRBASE"], "t", julia_zip)

g = request.urlopen(julia_url) 
with io.open(julia_zip_fullpath, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [ ]:
#checking it's there
assert os.path.isfile(julia_zip_fullpath)

In [ ]:
# checking the hashes
import hashlib
def give_hash(of_file, with_this):
    with io.open(julia_zip_fullpath, 'rb') as f:
        return with_this(f.read()).hexdigest()  
print (" "*12+"MD5"+" "*(32-12-3)+" "+" "*15+"SHA-256"+" "*(40-15-5)+"\n"+"-"*32+" "+"-"*64)

print ("%s %s %s" % (give_hash(julia_zip_fullpath, hashlib.md5) , give_hash(julia_zip_fullpath, hashlib.sha256),julia_zip))
assert give_hash(julia_zip_fullpath, hashlib.md5)  ==  hashes[0].lower() 
assert give_hash(julia_zip_fullpath, hashlib.sha256) ==  hashes[1].lower()

In [ ]:
# will be in env next time
os.environ["JUPYTER"] = os.path.join(os.environ["WINPYDIR"],"Scripts","jupyter.exe")
os.environ["JULIA_HOME"] = os.path.join(os.environ["WINPYDIRBASE"], "t", "julia-"+julia_version)
os.environ["JULIA_EXE_PATH"] = os.path.join(os.environ["JULIA_HOME"], "bin")
os.environ["JULIA_EXE"] = "julia.exe"
os.environ["JULIA"] = os.path.join(os.environ["JULIA_EXE_PATH"],os.environ["JULIA_EXE"])
os.environ["JULIA_PKGDIR"] = os.path.join(os.environ["WINPYDIRBASE"],"settings",".julia")
os.environ["JULIA_DEPOT_PATH"] = os.environ["JULIA_PKGDIR"] 
os.environ["JULIA_HISTORY"] = os.path.join(os.environ["JULIA_PKGDIR"],"logs","repl_history.jl")
os.environ["CONDA_JL_HOME"] = os.path.join(os.environ["JULIA_HOME"], "conda", "3")


# move JULIA_EXE_PATH to the beginning of PATH, since a julia installation may be present on the machine
os.environ["PATH"] = os.environ["JULIA_EXE_PATH"] + ";" + os.environ["PATH"]

In [ ]:
if not os.path.isdir(os.environ["JULIA_PKGDIR"]):
    os.mkdir(os.environ["JULIA_PKGDIR"])
    
if not os.path.isdir(os.path.join(os.environ["JULIA_PKGDIR"],"logs")):
    os.mkdir(os.path.join(os.environ["JULIA_PKGDIR"],"logs"))

In [ ]:
if not os.path.isfile(os.environ["JULIA_HISTORY"]):
    open(os.environ["JULIA_HISTORY"], 'a').close() # create empty file

In [ ]:
# extract the zip archive
import zipfile
try:
    with zipfile.ZipFile(julia_zip_fullpath) as z:
        z.extractall(os.path.join(os.environ["WINPYDIRBASE"], "t"))
        print("Extracted all files")
except:
    print("Invalid file")

In [ ]:
# delete zip file
os.remove(julia_zip_fullpath)

### 2 - Initialize Julia , IJulia, and make them link to winpython

In [ ]:
# connecting Julia to WinPython (only once, or everytime you move things)
# see the Windows terminal window for the detailed status. This may take 
# a minute or two.
import julia
julia.install()

In [ ]:
%load_ext julia.magic

In [ ]:
info = julia.juliainfo.JuliaInfo.load()
print(info.julia)
print(info.sysimage)
print(info.version_raw)

In [ ]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

In [ ]:
# sanity check
assert jl.eval("1+2") == 3

#### Print julia's versioninfo()
The environment should point to the usb drive and not to C:\\ (your local installation of julia maybe...)

In [ ]:
jl.eval("using InteractiveUtils")
jl.eval('file = open("julia_versioninfo.txt","w")')  
jl.eval("versioninfo(file,verbose=false)")
jl.eval("close(file)")

with open('julia_versioninfo.txt', 'r') as f:
    print(f.read())
    
os.remove('julia_versioninfo.txt')

#### Install julia Packages

In [ ]:
%%julia
using Pkg

Pkg.instantiate()
Pkg.update()

In [ ]:
%%julia
# add useful packages. Again, this may take a while...
Pkg.add("IJulia")
Pkg.add("Plots")
Pkg.add("Interact")
Pkg.add("Compose")
Pkg.add("SymPy")

using Compose
using SymPy
using IJulia
using Plots

#### Fix the kernel.json to allow arbitrary drive letters and modify the env.bat

the path to kernel.jl is hardcoded in the kernel.json file
this will cause trouble, if the drive letter of the usb drive changes
use relative paths instead
rewrite kernel.json and delete the one created from IJulia.jl Package

In [ ]:
kernel_path = os.path.join(os.environ["WINPYDIRBASE"], "settings", "kernels", "julia-"+julia_version[0:3])
assert os.path.isdir(kernel_path)

In [ ]:
with open(os.path.join(kernel_path,"kernel.json"), 'r') as f:
    kernel_str = f.read()

In [ ]:
new_kernel_str = kernel_str.replace(os.environ["WINPYDIRBASE"].replace("\\","\\\\"),"{prefix}\\\\..")
print(new_kernel_str)

In [ ]:
with open(os.path.join(kernel_path,"kernel.json"), 'w') as f:
    f.write(new_kernel_str)

In [ ]:
# add JULIA env variables to env.bat
inp_str = r"""
rem ******************
rem handle Julia {0} if included
rem ******************

if not exist "%WINPYDIRBASE%\t\julia-{0}\bin" goto julia_bad_{0}
set JULIA_PKGDIR=%WINPYDIRBASE%\settings\.julia
set JULIA_DEPOT_PATH=%JULIA_PKGDIR%
set JULIA_EXE=julia.exe
set JULIA_HOME=%WINPYDIRBASE%\t\julia-{0}
set JULIA_HISTORY=%JULIA_PKGDIR%\logs\repl_history.jl
:julia_bad_{0}

""".format(julia_version)

# append to env.bat
with open(os.path.join(os.environ["WINPYDIRBASE"],"scripts","env.bat"), 'a') as file :
    file.write(inp_str)

### 3 - Launching a Julia Notebook 

choose a Julia Kernel from Notebook, or Julia from Jupyterlab Launcher



### 4 - Julia Magic 
or use %load_ext julia.magic then %julia or %%julia